In [2]:
from fastapi import FastAPI
from pydantic import BaseModel
from gpt4all import GPT4All

# Phi-3 yolunu belirt (indirilen .gguf dosyasının yolu)
MODEL_PATH = r"C:\Users\erene\AppData\Local\nomic.ai\GPT4All\Phi-3-mini-4k-instruct.Q4_0.gguf"

app = FastAPI()

# Giriş veri modeli
class InputData(BaseModel):
    age: int
    sbp: int
    dbp: int
    hr: int
    rr: int
    bt: float
    mental: str
    sex: str
    arrival_mode: str
    injury: str
    pain: int
    chief_complaint: str

# Prompt formatı
def create_prompt(data):
    return f"""
Bir hasta acil servise başvurmuştur. Şikayeti: "{data['chief_complaint']}". Vital bulguları:

- Yaş: {data['age']}
- Sistolik Kan Basıncı (SBP): {data['sbp']} mmHg
- Diyastolik Kan Basıncı (DBP): {data['dbp']} mmHg
- Nabız (HR): {data['hr']} bpm
- Solunum Hızı (RR): {data['rr']} /dk
- Vücut Isısı (BT): {data['bt']} °C
- Bilinç Durumu: {data['mental']}
- Cinsiyet: {data['sex']}
- Geliş Şekli: {data['arrival_mode']}
- Travma: {data['injury']}
- Ağrı Seviyesi: {data['pain']} / 10

Yukarıdaki verilere göre bu hastanın KTAS (Acil Müdahale) seviyesini belirle.  
Cevabını sadece şu şekilde ver: Kırmızı, Sarı ya da Yeşil.
"""

# Phi-3 modeli belleğe al (ilk tahminde çağırmak için)
llm = GPT4All(model_name=MODEL_PATH, model_type="llama")

@app.post("/predict")
def predict(data: InputData):
    try:
        input_dict = data.dict()
        prompt = create_prompt(input_dict)

        with llm:
            output = llm.generate(prompt=prompt, max_tokens=100).strip()

        if "Kırmızı" in output:
            result = "Kırmızı"
        elif "Sarı" in output:
            result = "Sarı"
        elif "Yeşil" in output:
            result = "Yeşil"
        else:
            result = "Belirsiz"

        return {"ktas_tahmini": result, "model_cevabi": output}

    except Exception as e:
        return {"hata": str(e)}
